# Day 8

link: https://adventofcode.com/2022/day/8

ข้อนี่เริ่มน่าสนใจขึ้น เป็นข้อแรกที่เราเริ่มจะต้อง concern เกี่ยวกับ performance

ก่อนอื่นก็โหลด input เก็บไว้เป็นตัวเลขความสูงใน array 2 มิติ

In [1]:
input = IO.foreach('in08.txt').to_a.map(&:strip)

trees = input.map{|line| line.chars.map(&:to_i) }

rows = trees.size
cols = trees[0].size

99

## Part 1

โจทย์แรกให้เรานับจำนวนต้นไม้ที่มองเห็นได้จากภายนอก ตรงนี้ยังไม่ซับซ้อน เราก็ process ทีละทิศทาง จากทางซ้าย ขวา บน ล่าง
ไล่ดูต้นไม้ใน row/column นั้นๆ เก็บค่าต้นไม้ที่สูงที่สุดที่เคยเจอเอาไว้ ทุกครั้งที่เจอต้นที่สูงกว่าที่เคยเจอ ก็จะสรุปได้ว่าต้นนี้ visible
เราก็ mark ลง boolean array เก็บเอาไว้ แล้วไปนับตอนจบเมื่อวนครบทุกทิศทางแล้ว

ความท้าทายเล็กๆ ของโจทย์นี้คือจะเขียน logic ส่วนที่วนลูปเช็คความสูงแล้ว mark visibility ยังไงให้มัน reuse กันได้จากทั้ง 4 ทิศทาง
ซึ่งเราก็เขียนให้ `mark_visible` รับ coordinate ของต้นไม้ที่ต้องการเช็คเข้ามาทั้งแนว ก็คือทำให้ code นี้ไม่ขึ้นกับตำแหน่งทิศทางเลย
เราอยากจะเช็คแนว diagonal หรือจะสุ่มต้นไม้มาเช็คก็ยังได้

In [2]:
visible = rows.times.map{[false] * cols}

def mark_visible(trees, visible, cells)
  tallest = -1
  cells.each{|i, j|
    if trees[i][j] > tallest
      visible[i][j] = true
      tallest = trees[i][j]
    end
  }
end

rows.times{|i|
  cells = 0.upto(cols-1).map{|j| [i, j]}
  mark_visible(trees, visible, cells)
  mark_visible(trees, visible, cells.reverse)
}

cols.times{|j|
  cells = 0.upto(rows-1).map{|i| [i, j]}
  mark_visible(trees, visible, cells)
  mark_visible(trees, visible, cells.reverse)  
}

puts visible.map{|row| row.count{|x| x}}.sum

1543


## Part 2
 
โจทย์หลัง ให้เราหา maximum scenic score ซึ่งคำนวณจากผลคูณของต้นไม้ที่มองเห็นจากแต่ละทิศทาง

ถ้าคิดตรงๆ ใน grid ขนาด NxN สำหรับต้นไม้แต่ละต้นเราต้องเช็คทางซ้าย-ขวา O(N) ครั้ง เช็คทางบน-ล่างอีก O(N) ครั้ง
แล้วมันมีต้นไม้ทั้งหมด N^2 ต้น ดังนั้น complexity ของวิธีคิดตรงๆ นี้คือ O(N^3) ซึ่งสำหรับ N = 99 นี่มันก็ยังผ่านได้สบายๆ แหละ แต่เราทำได้ดีกว่านั้น

เราจะเก็บ score เป็น array คล้ายๆ กับ visibility array ใน Part 1 แต่คราวนี้เราให้ค่าเริ่มต้นเป็น 1 
จากนั้นจะเช็คทีละทิศทาง เอาจำนวนต้นไม้ที่มองเห็นได้คูณเข้าไปใน score array

วิธีหาจำนวนต้นไม้ที่มองเห็นได้ของแต่ละทิศทาง สมมติเป็นการมองไปทางซ้าย
เราให้ v_left(x, y, h) คือจำนวนต้นไม้ที่มองเห็นจากตำแหน่ง (x, y) ที่ความสูง h (เรา ignore ความสูงที่แท้จริงของต้นไม้ในตำแหน่งนั้น)
สังเกตว่า v_left(x, y, h) นั้นจะสัมพันธ์กับ v_left(x-1, y, h)
- ถ้าต้นไม้ (x, y) สูงกว่าหรือเท่ากับ h ก็จะเห็นได้แค่ (x-1, y) เพียงต้นเดียว ดังนั้น v_left(x, y, h) = 1
- ถ้าต้นไม่ (x, y) เตี้ยกว่า h เราก็จะเห็น (x-1, y) และเห็นต้นไม้ทุกต้นที่เห็นได้จาก (x-1, y, h) ดังนั้น v_left(x, y, h) = v_left(x-1, y, h) + 1

เราสามารถ loop จากซ้ายไปขวา คำนวณ v_left ในทุกๆ ความสูง h เก็บไว้ได้ ในแต่ละตำแหน่งจะมีค่าที่เอามาใช้คำนวณ scenic score จริงๆ 
ค่าเดียว ก็คือเมื่อ h เท่ากับความสูงของต้นไม้ในตำแหน่งนั้น แต่เราคำนวณทุกค่าเพื่อเอาไปใช้ในตำแหน่งถัดๆ ไปได้

วิธีนี้มี complexity เป็น O(h.N^2) โดยที่ h คือจำนวนความสูงที่เป็นไปได้

In [3]:
# All the border the score is actually 0, we'll just ignore them.
score = rows.times.map{[1] * cols}

def mult_score(trees, score, cells)
  visible = [1,1,1,1,1,1,1,1,1,1]
  cells.each{|i, j|
    score[i][j] *= visible[trees[i][j]]
    0.upto(9){|k|
      visible[k] = k > trees[i][j] ? visible[k] + 1 : 1
    }
  }
end

1.upto(rows-2){|i|
  cells = 1.upto(cols-2).map{|j| [i, j]}
  mult_score(trees, score, cells)
  mult_score(trees, score, cells.reverse)
}

1.upto(cols-2){|j|
  cells = 1.upto(rows-2).map{|i| [i, j]}
  mult_score(trees, score, cells)
  mult_score(trees, score, cells.reverse)  
}

puts score.map{|row| row.max}.max

595080
